# Processing solution SAXS data txt files from Tom

In [ ]:
# Imports

import pathlib
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from tqdm.auto import tqdm

In [ ]:
# Define paths

notebookPath = pathlib.Path.cwd()
outPath = notebookPath.parent.joinpath('processed_data/solution-saxs-5IDD')
txtsPath = outPath.joinpath('Acceptor_BHJ_SAXS')

In [ ]:
txt_files = sorted(txtsPath.glob('*txt'))

In [ ]:
[f.name for f in txt_files[1:]]

In [ ]:
samp_DS_rows = []
for txt_file in tqdm(txt_files[1:]):
    filename_split = txt_file.stem.split('_')
    
    data = np.loadtxt(txt_file)
    material = filename_split[0]
    solvent = filename_split[1]
    sample_name = material+'-'+solvent
    
    sol = filename_split[2]
    buf = filename_split[3]
    emp = filename_split[4]
    
    
    samp_DS = xr.Dataset(data_vars={'intensity': ('q', data[:, 1]),
                                    'error': ('q', data[:, 2])},
                         coords={'q':data[:,0],
                                 'sample_name':[sample_name], 
                                 'sol':('sample_name', [int(sol[3:])]),
                                 'buf':('sample_name', [int(buf[3:])]),
                                 'emp':('sample_name', [int(emp[3:])])})

    samp_DS_rows.append(samp_DS)
    
DS = xr.concat(samp_DS_rows, dim='sample_name')
DS

In [ ]:
DS.to_zarr(outPath.joinpath('acceptors.zarr'), mode='w')

In [ ]:
DS.sel(sample_name='Y7-CB')

In [ ]:
DS = xr.open_zarr(outPath.joinpath('acceptors.zarr')).compute()
DS

In [ ]:
for DA in DS['intensity']:
    DA.plot(xscale='log',yscale='log')
    plt.title(f'{DA.sample_name.values}, sol{DA.sol.values}')
    plt.show()

In [ ]:
# for txt_file in txt_files:
#     data = np.loadtxt(txt_file)
#     print(len(data[:,0]))
#     plt.plot(data[:,0], data[:,1])
#     plt.xscale('log')
#     plt.yscale('log')
#     plt.show()
#     plt.close('all')